In [103]:
import cloudscraper

In [175]:
scraper = cloudscraper.create_scraper()

In [173]:
import requests

In [182]:
headers = {
    'dnt': '1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'referer': 'https://www.amazon.ca/',
    'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
}


In [163]:
AMAZON_SEARCH_URL = 'https://www.amazon.ca/s?k='

In [106]:
search_query = 'iphone 15'
url = AMAZON_SEARCH_URL + '+'.join(search_query.split())

In [107]:
url

'https://www.amazon.ca/s?k=iphone+15'

In [108]:
result = scraper.get(url)

In [183]:
a = requests.get(url, headers)

In [184]:
a.status_code

503

In [109]:
result.status_code

503

In [110]:
result.text

'<!doctype html><html><head><meta charset="utf-8"><meta http-equiv="x-ua-compatible" content="ie=edge"><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"><title>Amazon.ca Something Went Wrong / Quelque chose s&#39;est mal passÃ©</title><style>html,body{padding:0;margin:0}img{border:0}#a,#b{background:#232f3e;padding:11px;height:35px}#c{position:absolute;left:22px;top:12px}#e{position:relative;max-width:800px;padding:0 40px 0 171px}#f,#g{height:35px;border:0;font-size:1em}#f{width:100%;margin:0;padding:0 10px;border-radius:4px 0 0 4px}#g{cursor:pointer;background:#febd69;font-weight:bold;border-radius:0 4px 4px 0;-webkit-appearance:none;position:absolute;top:0;right:0;padding:0 12px}@media(max-width:500px){#e{padding-left:0}#b{padding:55px 10px 10px}#c{left:6px}}#h{text-align:center;margin:30px 0}#h img{max-width:90%}#d{display:none}#d[src]{display:inline}</style></head><body><form id="b" accept-charset="utf-8" action="/s" method="GET" role="search"><a

In [111]:
html_page = result.text

In [188]:
from selenium import webdriver

In [189]:
driver = webdriver.Chrome()

url = 'https://www.amazon.ca/s?k=iphone+15'
driver.get(url)

In [190]:
driver.implicitly_wait(3)

In [191]:
page_source = driver.page_source

In [194]:
len(page_source)

1582349

In [192]:
driver.quit()

In [112]:
from bs4 import BeautifulSoup

In [121]:
with open('amazon-scraping-result.html', 'r') as f:
    html_page = f.read()

In [122]:
len(html_page)

1447238

In [123]:
html_page[:20]

'<!doctype html><html'

In [152]:
# for now, let's work with the saved version, then set up a scraper to obtain that html on its own
soup = BeautifulSoup(html_page, 'lxml')

In [161]:
# data-asin grabs products, while data-avar filters out sponsored ads
listings = soup.findAll('div', attrs={'data-asin': True, 'data-avar': False})

In [154]:
listings

[<div class="s-result-item s-widget s-widget-spacing-large AdHolder s-flex-full-width" data-asin="" data-index="0" data-uuid="b77a67e3-0177-490c-972a-d6a172ada727">
 <div cel_widget_id="lifestyle-image-v4-creative-desktop_loom-desktop-top-slot_1" class="celwidget pd_rd_w-fvRl1 content-id-amzn1.sym.8e0ef875-4e67-47f1-80ff-d43b41956e2f:amzn1.sym.8e0ef875-4e67-47f1-80ff-d43b41956e2f pf_rd_p-8e0ef875-4e67-47f1-80ff-d43b41956e2f pf_rd_r-4VNG9VAPAYRSRP9KTWZ6 pd_rd_wg-rzIwV pd_rd_r-309ea94c-ef9f-4b9a-b107-c40c3711aa40 c-f" data-csa-c-content-id="amzn1.sym.8e0ef875-4e67-47f1-80ff-d43b41956e2f:amzn1.sym.8e0ef875-4e67-47f1-80ff-d43b41956e2f" data-csa-c-painter="lifestyle-image-v4-creative-desktop-cards" data-csa-c-slot-id="DsUnknown-2" data-csa-c-type="widget" data-csa-op-log-render=""><script>if(window.mix_csa){window.mix_csa('[cel_widget_id="lifestyle-image-v4-creative-desktop_loom-desktop-top-slot_1"]', '#CardInstanceXd4s3ccFfdLGY66xpj86eg')('mark', 'bb')}</script>
 <script>if(window.uet){win

In [155]:
asin_values = [single_listing['data-asin'] for single_listing in listings if len(single_listing['data-asin']) != 0]

In [156]:
len(asin_values)

56

In [157]:
asin_values

['B0BN72FYFG',
 'B09HP4H4VG',
 'B0C5RC1C8C',
 'B08LDN5451',
 'B0CC1FTGMB',
 'B0CCJN8TY2',
 'B09LNP1ZHT',
 'B0BN91GD3J',
 'B0CGG3LGNW',
 'B08R6XGBTZ',
 'B0CKRQQ6K8',
 'B0BTYFSS6V',
 'B0CBBHX52Y',
 'B0CBV637MZ',
 'B0CHN26FJD',
 'B0CGCKL11P',
 'B0CFXH8LBV',
 'B0CCYM3F1V',
 'B0CBV7DRK7',
 'B0CRYR855L',
 'B09LP6WWLB',
 'B08PMKQP79',
 'B0CCCZ7MM9',
 'B0CJVBH96F',
 'B07WZSCWPJ',
 'B09LP77GDL',
 'B0CKM98MNW',
 'B0BLKJJCBV',
 'B0BDY71GRG',
 'B0C5S9WV88',
 'B0C89BY3L5',
 'B0C5S8XR9H',
 'B0CCYN42DL',
 'B0CJ6X6QP3',
 'B0CCNL2S29',
 'B0CDLVM18H',
 'B0CB6FHQRG',
 'B08PMYLKVF',
 'B09LPN7C8Z',
 'B0BN92S2ZZ',
 'B0CLLV6M2D',
 'B0C9Z34Q7F',
 'B09R7WFYWV',
 'B0CBB7HN82',
 'B0CBV6HD1M',
 'B0CJDMV6Q8',
 'B09YHNBYPS',
 'B0CB6DFQFK',
 'B0BDHB9Y8H',
 'B09G9F82WJ',
 'B0CHX6WQYM',
 'B0B3C774GQ',
 'B0C762Y69D',
 'B0C75ZRQLB',
 'B0CHX9FJWD',
 'B0BJMDHRPS']

In [166]:
most_relevant_asin = asin_values[0]

In [167]:
most_relevant_asin

'B0BN72FYFG'

In [162]:
# now, off to scrape for reviews

In [164]:
AMAZON_REVIEWS_URL = 'https://www.amazon.ca/product-reviews/'

In [168]:
current_url = AMAZON_REVIEWS_URL + most_relevant_asin

In [169]:
current_url

'https://www.amazon.ca/product-reviews/B0BN72FYFG'

In [172]:
star_page_suffix = {
    5: '/ref=cm_cr_unknown?filterByStar=five_star&pageNumber=1',
    4: '/ref=cm_cr_unknown?filterByStar=four_star&pageNumber=1',
    3: '/ref=cm_cr_unknown?filterByStar=three_star&pageNumber=1',
    2: '/ref=cm_cr_unknown?filterByStar=two_star&pageNumber=1',
    1: '/ref=cm_cr_unknown?filterByStar=one_star&pageNumber=1'
}